In [0]:
# Read the CSV file
df = spark.read.csv("/Volumes/workspace/default/spark_udemy/fakefriends.csv", inferSchema=True)

In [0]:
display(df)

In [0]:
df.createOrReplaceTempView("people")

In [0]:
%sql
select * from people

In [0]:
people = spark.read.option("header", "true").option("inferSchema", "true")\
    .csv("/Volumes/workspace/default/spark_udemy/fakefriends-header.csv")

In [0]:
display(people)

In [0]:
people.printSchema()

In [0]:
people.select("name").show()

In [0]:
people.filter(people.age < 21).show()

In [0]:
people.groupBy('age').count().show()

In [0]:
people.select(people.name, people.age + 10).show()

In [0]:
people.groupBy("age").avg("friends").show()

In [0]:
people.count()

In [0]:
from pyspark.sql import functions as func

people.groupBy("age").agg(func.round(func.avg("friends"), 2)).sort("age").show()

In [0]:
df = spark.read.text("/Volumes/workspace/default/spark_udemy/book.txt")

In [0]:
display(df)

In [0]:
df = df.select(func.explode(func.split(df.value, "\\W+")).alias("word"))
display(df)

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

schema = StructType([ \
                     StructField("stationID", StringType(), True), \
                     StructField("date", IntegerType(), True), \
                     StructField("measure_type", StringType(), True), \
                     StructField("temperature", FloatType(), True)])

In [0]:
df = spark.read.schema(schema).csv("/Volumes/workspace/default/spark_udemy/1800.csv")
df.head()

In [0]:
display(df)

In [0]:
# Filter out all but TMIN entries
minTemps = df.filter(df.measure_type == "TMIN")

In [0]:
# Select only stationID and temperature
stationTemps = minTemps.select("stationID", "temperature")

# Aggregate to find minimum temperature for every station
minTempsByStation = stationTemps.groupBy("stationID").min("temperature")
display(minTempsByStation)
minTempsByStation.withC

In [0]:
minTempsByStationF = minTempsByStation.withColumn("temperature",
                                                  func.round(func.col("min(temperature)") * 0.1 * (9.0 / 5.0) + 32.0, 2))\
                                                  .select("stationID", "temperature").sort("temperature")

In [0]:
display(minTempsByStationF)